In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
import itertools

In [2]:
# to provide easy switching between cpu and gpu
CUDA = print(torch.cuda.is_available())
device = torch.device("cuda" if CUDA else "cpu")

False


# Part 1 :  **Data Preprocessing**

In [3]:
with open('movie_conversations.txt','r') as conv_file:
  convs = conv_file.readlines()
for conv in convs[:10]:
  print(conv.strip())

u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']
u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']


In [4]:
with open('movie_lines.txt', 'r', errors='replace') as lines_file:
  lines = lines_file.readlines()
for line in lines[:10]:
  print(line.encode('utf-8').strip())

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go."
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie."
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?'


In [5]:
line_fields = ['lineID', 'charID', 'movieID', 'char_name', 'text']
lines = {}
with open('movie_lines.txt','r',encoding='iso-8859-1') as file: #encodes ASCII chars, single Byte encoding method
#   head = [next(file) for x in range(10)]
  for line in file:
    vals = line.strip().split('+++$+++')
    lineID = vals[0].strip(' ')
    line_obj = {}
    line_obj['lineID'] = lineID
    for i in range(1,len(vals)):
      line_obj[line_fields[i]] = vals[i]
    lines[line_obj['lineID']] = line_obj

In [6]:
import pprint

In [7]:
conv_fields = ['char1ID', 'char2ID', 'movieID', 'dialoguesIDs']
convs = []
with open('movie_conversations.txt', 'r', encoding='iso-8859-1') as file:
  for line in file:
    vals = line.strip().split('+++$+++')
    conv_obj = {}
    for i in range(len(vals)):
      conv_obj[conv_fields[i]] = vals[i]
    lineIDs = conv_obj['dialoguesIDs'] # get line IDs from dialoguesIDs string, string type
    lineIDs = lineIDs[2:-1] # remove braces from string 
    conv_obj["lines"] = []
    
    for lineID in lineIDs.split(','): # split line IDs separated by ','
      lineID = lineID.strip('\' ')
      try:
        conv_obj["lines"].append(lines[lineID]) #search for that line in lines dict with this linsID
      except KeyError:
        not_found += 1
#         print("here")
        not_found.append(lineID)
    convs.append(conv_obj)
pp = pprint.PrettyPrinter()
pp.pprint(convs[:10])

[{'char1ID': 'u0 ',
  'char2ID': ' u2 ',
  'dialoguesIDs': " ['L194', 'L195', 'L196', 'L197']",
  'lines': [{'charID': ' u0 ',
             'char_name': ' BIANCA ',
             'lineID': 'L194',
             'movieID': ' m0 ',
             'text': ' Can we make this quick?  Roxanne Korrine and Andrew '
                     'Barrett are having an incredibly horrendous public '
                     'break- up on the quad.  Again.'},
            {'charID': ' u2 ',
             'char_name': ' CAMERON ',
             'lineID': 'L195',
             'movieID': ' m0 ',
             'text': " Well, I thought we'd start with pronunciation, if "
                     "that's okay with you."},
            {'charID': ' u0 ',
             'char_name': ' BIANCA ',
             'lineID': 'L196',
             'movieID': ' m0 ',
             'text': ' Not the hacking and gagging and spitting part.  '
                     'Please.'},
            {'charID': ' u2 ',
             'char_name': ' CAMERON ',
 

In [8]:
print(len(convs))

83097


In [9]:
#extract pairs of conversations
conv_pairs = []
for conv in convs :
    for i in range(len(conv['lines'])-1):
        conv1 = conv['lines'][i]['text']
        conv2 = conv['lines'][i+1]['text']
        if conv1 and conv2:
            conv_pairs.append([conv1, conv2])
print(len(conv_pairs))

221282


In [10]:
#create new file containing conversation pairs
delimiter = '\t'
delimiter = str(codecs.decode(delimiter, "unicode_escape"))
with open("conv_pairs.txt", "w", encoding='utf-8') as out_file:
    writer = csv.writer(out_file, delimiter=delimiter)
    for pair in conv_pairs:
        writer.writerow(pair)

In [11]:
with open('conv_pairs.txt','rb') as file:
    lines = file.readlines()
    print(len(lines))
    for line in lines[:10]:
        print(line)

221282
b" Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\t Well, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b" Well, I thought we'd start with pronunciation, if that's okay with you.\t Not the hacking and gagging and spitting part.  Please.\r\n"
b" Not the hacking and gagging and spitting part.  Please.\t Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b" You're asking me out.  That's so cute. What's your name again?\t Forget it.\r\n"
b" No, no, it's my fault -- we didn't have a proper introduction ---\t Cameron.\r\n"
b" Cameron.\t The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b" The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\t Seems like she could get a date easy enough...\r\n"
b'

Part 2 : **Processing the words**

In [12]:
#turn unicode string to plain ascii

# NFD : Normal Form Decomposed - decomposes the chars and combines it together
# Mn : none marking space
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD',s) if unicodedata.category(c) != 'Mn')

In [27]:
def normalise_string(s):
    s = unicode_to_ascii(s.lower().strip())
    
    # replace any char(?!.) by whitespace + char
    # \1 means first bracketed group --> [?!.]
    # r is not to consider \1 as a character
    # r to escape backslash
    s = re.sub(r"([?!.])", r" \1",s)
    
    # remove any char that's not a sequence of  [a-zA-z?!.] 
    s = re.sub(r"[^a-zA-z?!.]+", r" ", s)
    
    # remove sequence of whitespace chars
    s = re.sub(r"\s+", r" ", s)
    
    return s

In [28]:
normalise_string("frjf45  ? !.")

'frjf ? ! .'

In [29]:
datafile = 'conv_pairs.txt'

# list 'lines' elements are conversation pairs
lines = open(datafile, encoding='utf-8').read().strip().split('\n')
print("reading and processing file")

# for each dialogue in the pair, normalise the dialogue string
# pairs of normalised strings are obtained
pairs = [[normalise_string(s) for s in pairs.split('\t')] for pairs in lines]

print("done processing")

reading and processing file
done processing


In [30]:
pairs[:10]

[['can we make this quick ? roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad . again .',
  'well i thought we d start with pronunciation if that s okay with you .'],
 ['well i thought we d start with pronunciation if that s okay with you .',
  'not the hacking and gagging and spitting part . please .'],
 ['not the hacking and gagging and spitting part . please .',
  'okay . . . then how bout we try out some french cuisine . saturday ? night ?'],
 ['you re asking me out . that s so cute . what s your name again ?',
  'forget it .'],
 ['no no it s my fault we didn t have a proper introduction ', 'cameron .'],
 ['cameron .',
  'the thing is cameron i m at the mercy of a particularly hideous breed of loser . my sister . i can t date until she does .'],
 ['the thing is cameron i m at the mercy of a particularly hideous breed of loser . my sister . i can t date until she does .',
  'seems like she could get a date easy enough . . .'],
 ['why 

In [31]:
len(pairs)

221282

In [32]:
# keep conversation pairs where dialogue length <= 10 words, check both dialogues

MAX_LENGTH = 10
def filter_pairs(s):
    # check that both dialogues' length < 2= 10
    return len(s[0].split()) <= MAX_LENGTH and len(s[1].split()) <= MAX_LENGTH

# keep only those pairs that return true for above function
pairs = [pair for pair in pairs if filter_pairs(pair)]

In [33]:
len(pairs)

75024

In [34]:
# takes 'pairs' - list of pair of dialogues,return max length of any dialogue in pairs
def max_len_of_dialogue(pairs):
    max_len = 0
    for pair in pairs[:10]:
        p1 = pair[0].split()
        p2 = pair[1].split()
        l1 = len(p1)
        l2 = len(p2)
        max_len = l1 if l1 > max_len else l2 if l2 > max_len else max_len
    return max_len

In [35]:
pairs = [pair for pair in pairs if len(pair)>1]

In [36]:
len(pairs)

75024

In [37]:
PAD_token = 0
SOS_token = 1
EOS_token = 2

class Vocabulary():
    def __init__(self, name):
        self.name = name # assign name to different vocabularies, if there are > 1
        self.word2ind = {} # dictionary to store unique index for each word in dictionary{word:id}
        self.word2count = {} # store frequency of each word
        self.ind2word = {PAD_token:"PAD", SOS_token:"SOS", EOS_token:"EOS"}  #id:word
        self.num_words = 3 #PAD, SOS, EOS
    
    def word_count(self, sentence): # function to count word frequency and add word in vocabulary
        for word in sentence.split(' '):
            try: # if the word exists, increment its count
                self.word2count[word] += 1 
            except KeyError: # if the word doesnot already exist in vocabulary
                self.word2count[word] = 1
                #self.add_word(word)
    
    def add_word(self, word):
        self.word2ind[word] = self.num_words 
        self.ind2word[self.num_words] = word
        self.num_words += 1
            
    def __str__(self):
        return "Vocabulary size: {}".format(str(self.num_words))
    
    # remove ethose words from vocabulary whose frequency is below certain threshold
    def trim(self, threshold):
        deleted_words = 0
        for k,v in self.word2count.items():
            if v < threshold:
                deleted_words += 1
                self.word2count[k]= 0 # set frequency of word in word2count dict = 0
            else:
                self.add_word(k)
        #for word in self.word2count.keys():
         #   if self.word2count[word] > 0:
          #      self.add_word(word)
        
        words_in_vocab = len(self.word2ind)
        words_in_corpus = words_in_vocab + deleted_words
    
        print("keep words {}/{} = {:.4f}".format(words_in_vocab, words_in_corpus, words_in_vocab/words_in_corpus ))

In [38]:
vocab = Vocabulary("cornell-movie-dialogues-corpus")
MIN_FREQ = 3 # only keep words with frequqncy >= 3

In [39]:
# add words to vocab whose frequency >= 3
for pair in pairs:
    vocab.word_count(pair[0])
    vocab.word_count(pair[1])
    
vocab.trim(MIN_FREQ)

keep words 9045/20173 = 0.4484


In [40]:
len(pairs)

75024

In [41]:
def remove_pairs(): # remove those pairs whose any of the 2 dialogues contains infrequent words
    keep_pairs = []
    
    for pair in pairs:
        keep_first = True # flag to keep or remove 1st dialogue
        keep_sec = True# flag to keep or remove 2nd dialogue
        for word in pair[0].split(): # 1st dialogue
            if word not in vocab.word2ind:
                keep_first = False
                break
        for word in pair[1].split(): # 2nd dialogue
            if word not in vocab.word2ind:
                keep_sec = False
                break
        if keep_first and keep_sec :
             keep_pairs.append(pair)
    return keep_pairs
    #print("pairs earlier : {}, pairs now: {}".format(len(pairs),len(keep_pairs)))

pairs = remove_pairs()

In [42]:
print(len(pairs))

62751


In [43]:
def indexes_from_sentences(voc_obj, sentence):
    return [voc_obj.word2ind[word] for word in sentence.split()]+[EOS_token]

In [44]:
print(pairs[60000][0])
print(indexes_from_sentences(vocab, pairs[60000][0]))

burt reynolds .
[7045, 3376, 11, 2]


In [45]:
vocab.word2ind['interested']

1277

In [46]:
print(vocab)

Vocabulary size: 9048


In [47]:
inp = []
for pair in pairs[:10]:
    inp.append(pair[0])
print(inp)
ind = [indexes_from_sentences(vocab, sentence) for sentence in inp]
print(ind)

['that s because it s such a nice one .', 'there .', 'you have my word . as a gentleman', 'hi .', 'have fun tonight ?', 'well no . . .', 'then that s all you had to say .', 'but', 'do you listen to this crap ?', 'what good stuff ?']
[[3, 4, 5, 6, 4, 7, 8, 9, 10, 11, 2], [14, 11, 2], [17, 18, 19, 20, 11, 21, 8, 22, 2], [25, 11, 2], [18, 40, 31, 16, 2], [42, 43, 11, 11, 11, 2], [44, 3, 4, 45, 17, 46, 47, 48, 11, 2], [49, 2], [54, 17, 55, 47, 52, 56, 16, 2], [57, 58, 59, 16, 2]]


In [48]:
vocab.ind2word[2]

'EOS'

In [49]:
a = ['a', 'b', 'c', 'd', 'e']
b = [1,2,3]
c = itertools.zip_longest(a,b)
c
#list(c) # an iterator is returned which is then converted to list

In [50]:
next(c) #

('a', 1)

In [51]:
list(c)

[('b', 2), ('c', 3), ('d', None), ('e', None)]

In [52]:
list(itertools.zip_longest(*ind, fillvalue=0)) # zip values colummn-wise, fill 0 at places where None 

[(3, 14, 17, 25, 18, 42, 44, 49, 54, 57),
 (4, 11, 18, 11, 40, 43, 3, 2, 17, 58),
 (5, 2, 19, 2, 31, 11, 4, 0, 55, 59),
 (6, 0, 20, 0, 16, 11, 45, 0, 47, 16),
 (4, 0, 11, 0, 2, 11, 17, 0, 52, 2),
 (7, 0, 21, 0, 0, 2, 46, 0, 56, 0),
 (8, 0, 8, 0, 0, 0, 47, 0, 16, 0),
 (9, 0, 22, 0, 0, 0, 48, 0, 2, 0),
 (10, 0, 2, 0, 0, 0, 11, 0, 0, 0),
 (11, 0, 0, 0, 0, 0, 2, 0, 0, 0),
 (2, 0, 0, 0, 0, 0, 0, 0, 0, 0)]

In [53]:
def zero_padding(l, fillvalue=0): # default padding is with 0s
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

In [54]:
print(ind)

[[3, 4, 5, 6, 4, 7, 8, 9, 10, 11, 2], [14, 11, 2], [17, 18, 19, 20, 11, 21, 8, 22, 2], [25, 11, 2], [18, 40, 31, 16, 2], [42, 43, 11, 11, 11, 2], [44, 3, 4, 45, 17, 46, 47, 48, 11, 2], [49, 2], [54, 17, 55, 47, 52, 56, 16, 2], [57, 58, 59, 16, 2]]


In [55]:
# max length will be 11 as EOS token is also appended at the end of each sentence
leng = [len(index) for index in ind]
max(leng) 

11

In [56]:
padded = zero_padding(ind) # in padded, max_len(11) is now the number of rows, #cols = batch size
print(len(padded))
padded

11


[(3, 14, 17, 25, 18, 42, 44, 49, 54, 57),
 (4, 11, 18, 11, 40, 43, 3, 2, 17, 58),
 (5, 2, 19, 2, 31, 11, 4, 0, 55, 59),
 (6, 0, 20, 0, 16, 11, 45, 0, 47, 16),
 (4, 0, 11, 0, 2, 11, 17, 0, 52, 2),
 (7, 0, 21, 0, 0, 2, 46, 0, 56, 0),
 (8, 0, 8, 0, 0, 0, 47, 0, 16, 0),
 (9, 0, 22, 0, 0, 0, 48, 0, 2, 0),
 (10, 0, 2, 0, 0, 0, 11, 0, 0, 0),
 (11, 0, 0, 0, 0, 0, 2, 0, 0, 0),
 (2, 0, 0, 0, 0, 0, 0, 0, 0, 0)]

In [57]:
# function to find actual words in list l, if padded_token found(value 0),put 0, else 1
def binary_padding(l):
    m = []
    for i,seq in enumerate(padded):
        m.append([])
        for ind in seq:
            if ind == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

In [58]:
binary_padding(padded)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 0, 1, 1, 0, 1, 0],
 [1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
 [1, 0, 1, 0, 0, 0, 1, 0, 1, 0],
 [1, 0, 1, 0, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [59]:
def input_var(l, vocab):
    indexes_batch = [indexes_from_sentences(vocab, sentence) for sentence in l]
    lengths = torch.tensor([len(ind) for ind in indexes_batch])
    pad_list = zero_padding(indexes_batch)
    pad_var = torch.LongTensor(pad_list)
    return pad_var, lengths

In [60]:
def output_var(l, vocab):
    indexes_batch = [indexes_from_sentences(vocab, sentence) for sentence in l]
    max_target_len = max([len(ind) for ind in indexes_batch])
    pad_list = zero_padding(indexes_batch)
    mask = torch.ByteTensor(binary_padding(pad_list))
    pad_var = torch.LongTensor(pad_list)
    return max_target_len, mask, pad_var

In [61]:
def batch2train_data(batch_pairs, vocab):
    
    # arrange pairs in decreasing order of length of 1st dialogue(question)
    batch_pairs.sort(key = lambda x : len(x[0].split()), reverse=True)
    ques_batch,replies_batch = [],[]
    for pair in batch_pairs:
        ques_batch.append(pair[0])
        replies_batch.append(pair[1])
    inp, lengths = input_var(ques_batch, vocab)
    max_target_len, mask, output = output_var(replies_batch, vocab)
    return inp, lengths, max_target_len, mask, output

In [62]:
# test above function with a small batch size
bs = 5

# randomly select 5 dialogue pairs from 'pairs'
pairs = [random.choice(pairs) for _ in range(bs)]
print(pairs)
inp, lengths, max_target_len, mask, output = batch2train_data(pairs,vocab) 
print("inp", inp)
print("lengths", lengths)
print("max_target_len", max_target_len)
print("mask", mask)
print("output", output)

[['what are you going to do ? hm ?', 'if you hit me . . .'], ['do you still want the operation ?', 'yeah . . . yeah .'], ['good . see ya soon .', 'good luck .'], ['any brothers ? sisters ?', 'dick jane and spot .'], ['what s the matter ?', 'you fixed this suit ? beth ?']]
inp tensor([[  57,   54,   58,  236,   57],
        [ 112,   17,   11, 3890,    4],
        [  17,  362,  394,   16,   61],
        [ 149,  144,  239, 3973,  372],
        [  47,   61, 1238,   16,   16],
        [  54,  760,   11,    2,    2],
        [  16,   16,    2,    0,    0],
        [2674,    2,    0,    0,    0],
        [  16,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths tensor([10,  8,  7,  6,  6])
max_target_len 8
mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 1, 1],
        [1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
        [1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
        [1, 0, 1, 0, 0, 1, 1, 0, 1, 0],
        [1, 0, 1, 0, 0,

# Part 2 : Defining the models

Encoder in GRU only gives one output - the hidden state output. 

    For encoder:
    Steps to be taken are : convert word indexes to embeddings, Pack padded batch of sequences for RNN module, Forward pass through GRU, Unpack padding, Finally return output and final hidden state.

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.n_layers = n_layers
        
        #the input is word embedding whose size is that of hidden layer
        #arguments - input size, hidden size, n_layers, dropout, bidirectional(default value-False)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers=1 else droput))
        
    def forward(self, inp_seq, inp_length, hidden=None):
        # inp_seq shape(which will be pack padded seq) = (batch_size,max_len)
        # inp_lengths : length of all input sentences in batch
        # hidden(n_layers * num_directions, batch, hidden_size) : initial hidden state for each element in batch
        
        #convert word indexes to embeddings
        embedded = self.embedding(inp_seq)
        
        #pack padded batch of sequences for RNN module
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded)
        
        #forward pass through GRU
        output, hidden = self.gru(packed, hidden)
        
        #unpack padding
        output, _ = torch.nn.utils.rnn.pad_packed_sequence(output)
        
        #return final output and final hidden state
        return output, hidden